# Median House Value Assesment Activity

This California Housing Prices dataset has been downloaded from StatLib repository (http://lib.stat.cmu.edu/datasets/). It is based on data from the 1990 California census, what is not important for deep learning. The original dataset appeared in R. Kelley Pace and Ronald Barry, “Sparse Spatial Autoregressions,” Statistics & Probability Letters 33, no. 3 (1997): 291–297.

<b>MedianHouseValuePreparedCleanAttributes.csv</b><br>The original dataset contained 20,640 instances, which is cleaned, preprocessed and prepared in this notebook. After this phase of data preparation, a final dataset of 20,433 instances are obtained with 8 attributes individually normalized with a min-max scaling, $\frac{x-min}{max-min}$ (InputsMedianHouseValueNormalized.csv): $longitude$ and $latitude$ (location), $median age$, $total rooms$, $total bedrooms$, $population$, $households$ and $median income$.  

From this data, the classification problem consists on estimating the median house value, categorized into the following 10 clases (price intervals in thousand dollards): [15.0, 82.3], [82.4, 107.3], [107.4, 133.9], [134.0, 157.3], [157.4, 179.7], [179.8, 209.4], [209.5, 241.9], [242.0, 290.0], [290.1, 376.6] and [376.7, 500.0]. Each class is labelled from 0 (the cheapest) to 9 (the most expensive), and one-hot encoded in <b>MedianHouseValueOneHotEncodedClasses.csv</b> file.

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from google.colab import files
uploaded = files.upload()


Saving MedianHouseValueOneHotEncodedClasses.csv to MedianHouseValueOneHotEncodedClasses.csv
Saving MedianHouseValuePreparedCleanAttributes.csv to MedianHouseValuePreparedCleanAttributes.csv
Saving OceanProximityPreparedCleanAttributes.csv to OceanProximityPreparedCleanAttributes.csv


In [10]:
%run 1.ReadingData.py

x_train: (16346, 8)
t_train: (16346, 10)
x_dev: (2043, 8)
t_dev: (2043, 10)
x_test: (2044, 8)
t_test: (2044, 10)


## Initialization

In [0]:
INPUTS = x_train.shape[1]
OUTPUTS = t_train.shape[1]
NUM_TRAINING_EXAMPLES = int(round(x_train.shape[0]/1))
NUM_DEV_EXAMPLES = int (round (x_dev.shape[0]/1))
NUM_TEST_EXAMPLES = int (round (x_test.shape[0]/1))

Some data is displayed to test the correctness:

In [12]:
INPUTS #Should be 8

8

In [13]:
OUTPUTS #Should be 10

10

In [14]:
NUM_TRAINING_EXAMPLES #16346

16346

In [15]:
NUM_DEV_EXAMPLES #2043

2043

In [0]:
NUM_TEST_EXAMPLES #2044

In [0]:
x_train[:5]

In [0]:
t_train[:5]

In [0]:
x_dev[:5]

In [0]:
t_dev[:5]

In [0]:
x_test[:5]

In [0]:
t_test[:5]

In [16]:
print(type(x_train))
print('Training shape:', x_train.shape)
print('Training samples: ', x_train.shape[0])
print('Validation samples: ', x_test.shape[0])

<class 'numpy.ndarray'>
Training shape: (16346, 8)
Training samples:  16346
Validation samples:  2044


## Hyperparameters

Some hyperparameters given as example (they may not be the right ones):

In [0]:
n_epochs = 5000 
batch_size = 200
n_neurons_per_layer = [80,100,40,10] 

In [0]:
X = tf.placeholder (dtype=tf.float32, shape=(None,INPUTS),name="X")
t = tf.placeholder (dtype=tf.float32, shape=(None,OUTPUTS), name="t")

In [19]:
hidden_layers = []
hidden_layers.append(tf.layers.dense (X, n_neurons_per_layer[0], 
                                      activation = tf.nn.relu))
for layer in range(1,len(n_neurons_per_layer)):
    hidden_layers.append(tf.layers.dense (hidden_layers[layer-1], 
                 n_neurons_per_layer[layer], activation = tf.nn.relu))
net_out = tf.layers.dense (hidden_layers[len(n_neurons_per_layer)-1], OUTPUTS)
y = tf.nn.softmax (logits=net_out, name="y")

Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [23]:
for layer in range(len(n_neurons_per_layer)): print (hidden_layers[layer])

Tensor("dense/Relu:0", shape=(?, 80), dtype=float32)
Tensor("dense_1/Relu:0", shape=(?, 100), dtype=float32)
Tensor("dense_2/Relu:0", shape=(?, 40), dtype=float32)
Tensor("dense_3/Relu:0", shape=(?, 10), dtype=float32)


In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2 (labels=t, logits=net_out)
mean_log_loss = tf.reduce_mean (cross_entropy, name="cost")

In [22]:
train_step = tf.train.GradientDescentOptimizer (learning_rate).minimize(mean_log_loss)

NameError: ignored

In [0]:
correct_predictions = tf.equal(tf.argmax(y,1),tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions,tf.float32))

In [0]:
initial_learning_rate = 0.1
decay_steps = 10000
decay_rate = 0.96
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(initial_learning_rate,global_step, decay_steps,decay_rate)

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9,use_nesterov=True)
#optimizer = tf.train.RMSPropOptimizer(learning_rate= 0.1,decay=0.9, epsilon=1e-10)
train_step = optimizer.minimize(mean_log_loss,global_step=global_step)

In [26]:
init = tf.global_variables_initializer()
accuracy_train_history = []
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(n_epochs)):
        offset = (epoch * batch_size) % (NUM_TRAINING_EXAMPLES - batch_size)
        sess.run (train_step, feed_dict={X: x_train[offset:(offset+batch_size)],
                                         t: t_train[offset:(offset+batch_size)]})
            
    accuracy_test = accuracy.eval(feed_dict={X: x_test[:NUM_TEST_EXAMPLES],
                                              t: t_test[:NUM_TEST_EXAMPLES]})
    test_predictions = y.eval(feed_dict={X: x_test[:NUM_TEST_EXAMPLES]})
    
    test_correct_preditions = correct_predictions.eval (feed_dict=
                                    {X: x_test[:NUM_TEST_EXAMPLES],
                                     t: t_test[:NUM_TEST_EXAMPLES]}
    )   


100%|██████████| 5000/5000 [00:10<00:00, 475.50it/s]


In [27]:
"Accuracy for the TEST set: " + str(accuracy_test) 

'Accuracy for the TEST set: 0.4163405Tensor("Equal:0", shape=(?,), dtype=bool)'

In [0]:
tensor_board = TensorBoard('./logs')

In [0]:
test_predictions

In [0]:
test_rounded_predictions=np.round(test_predictions)
indices = np.argmax(test_predictions,1)
for row, index in zip(test_rounded_predictions, indices): row[index]=1
test_rounded_predictions[:10]

In [0]:
test_correct_preditions[:10]